In [1]:
import sys
sys.path.append('../')
import btax_mini
from btax_mini.run_btax import run_btax, run_btax_with_baseline_delta
import taxcalc
from taxcalc import *
import pandas as pd
import copy
import numpy as np
import math
ctax_data_path = 'aggregates_data/'

# Functions and base values

## Import base corporate tax data

In [2]:
taxrev_data = pd.read_csv(ctax_data_path + 'taxrev.csv')

## AMT and PYMTC model

In [3]:
#import data and set parameter values
amtdata2 = pd.read_csv(ctax_data_path + 'amt data2.csv')
#theta_set = [0.3914, 0.9679249, 1] #usable portion of PYMTC stock, estimated separately
theta_set = [0.269045, 0.920237, 1]

In [4]:
def calcAMTparams():
    amt_data2 = copy.deepcopy(amtdata2)
    Clist = amt_data2['C'].tolist()
    Alist = amt_data2['A'].tolist()[:20]
    Plist = amt_data2['P'].tolist()[:20]
    Slist = np.zeros(len(Alist))
    Slist[19] = 26.0
    AMT_rate = 0.2
    Ctax_rate = 0.35
    for i in range(19):
        Slist[18-i] = Slist[19-i] + Plist[18-i] - Alist[18-i]
    gross_use_rate = sum([Plist[i] / Slist[i] for i in range(20)]) / 20.
    gamma = gross_use_rate / theta_set[0]
    nu = sum([Alist[i] / Clist[i] for i in range(7,20)]) / 13. * Ctax_rate / AMT_rate
    return (gamma, nu)
(gamma, nu) = calcAMTparams()

def AMTmodel(amt_repeal_year=9e99, pymtc_repeal_year=9e99, amt_rateChange_year=9e99, ctax_rateChange_year=9e99, 
             amt_newRate=0.2, ctax_newRate=0.347):
    assert min(amt_repeal_year, pymtc_repeal_year, amt_rateChange_year, ctax_rateChange_year) >= 2014
    assert amt_newRate >= 0
    assert ctax_newRate >= 0
    AMT_rate = 0.2
    Ctax_rate = 0.347
    amt_data2 = copy.deepcopy(amtdata2)
    Clist = amt_data2['C'].tolist()[19:]
    Alist = [amt_data2['A'][19]]
    Plist = [amt_data2['P'][19]]
    Slist = [26.0]
    Slist.append(Slist[0] + Alist[0] - Plist[0])
    for i in range(1,15):
        # Determine tax rates
        if i + 2013 >= amt_rateChange_year:
            AMT_rate = amt_newRate
        if i + 2013 >= ctax_rateChange_year:
            Ctax_rate = ctax_newRate
        # Determine AMT revenue
        if i + 2013 >= amt_repeal_year:
            Alist.append(0)
        else:
            Alist.append(Clist[i] * nu * min(AMT_rate / Ctax_rate, 1))
        ## Determine PYMTC
        if i + 2013 >= pymtc_repeal_year:
            Plist.append(0)
        elif i + 2013 < amt_repeal_year:
            Plist.append(Slist[i] * gamma * theta_set[0])
        elif i + 2013 == amt_repeal_year:
            Plist.append(Slist[i] * gamma * theta_set[1])
        else:
            Plist.append(Slist[i] * gamma * theta_set[2])
        Slist.append(Slist[i] + Alist[i] - Plist[i])
    AMT_results = pd.DataFrame({'year': range(2014,2028), 'amt': Alist[1:], 'pymtc': Plist[1:]})
    return AMT_results

amt_base = AMTmodel()

## Foreign Tax Credit model

In [5]:
ftc_taxrates_data = pd.read_csv(ctax_data_path + 'ftc taxrates data.csv')
ftc_gdp_data = pd.read_csv(ctax_data_path + 'ftc gdp data.csv')
ftc_other_data = pd.read_csv(ctax_data_path + 'ftc other data.csv')

def calcWAvgTaxRate(year):
    assert year in range(1995,2028)
    if year > 2016:
        year = 2016
    gdp_list = np.asarray(ftc_gdp_data[str(year)])
    taxrate_list = np.asarray(ftc_taxrates_data[str(year)])
    avgrate = sum(np.where(np.isnan(taxrate_list), 0, taxrate_list * gdp_list)) / sum(np.where(np.isnan(taxrate_list), 0, gdp_list))
    return avgrate

def calcFTCAdjustment():
    ftc_actual = np.asarray(ftc_other_data['F'][:19])
    profits = np.asarray(ftc_other_data['C_total'][:19])
    profits_d = np.asarray(ftc_other_data['C_domestic'][:19])
    profits_f = profits - profits_d
    tax_f = []
    for i in range(1995,2014):
        tax_f.append(calcWAvgTaxRate(i))
    ftc_gross = profits_f * tax_f / 100.
    adjfactor = sum(ftc_actual / ftc_gross) / 19.
    return adjfactor

adjfactor_ftc = calcFTCAdjustment()

def FTC_model(haircut=0.0, haircut_year = 9e99):
    profits = np.asarray(ftc_other_data['C_total'][19:])
    profits_d = np.asarray(ftc_other_data['C_domestic'][19:])
    tax_f = np.zeros(14)
    hc_applied = np.zeros(14)
    for i in range(14):
        tax_f[i] = calcWAvgTaxRate(i + 2014)
        if i + 2014 >= haircut_year:
            hc_applied = haircut
    ftc_final = (profits - profits_d) * tax_f / 100. * adjfactor_ftc * (1 - hc_applied)
    ftc_results = pd.DataFrame({'year': range(2014,2028), 'ftc': ftc_final})
    return ftc_results

ftc_base = FTC_model()

## Taxable Income

In [6]:
combined_base = taxrev_data.merge(right=amt_base, how='outer', on='year')
combined_base = combined_base.merge(right=ftc_base, how='outer', on='year')
combined_base['taxbc'] = combined_base['taxrev'] + combined_base['pymtc'] + combined_base['ftc'] - combined_base['amt']
combined_base['gbc_adj'] = 0.025739617
combined_base['tau'] = 0.347
combined_base['taxinc'] = combined_base['taxbc'] / (combined_base['tau'] - combined_base['gbc_adj'])
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc
0,2014,320.731,4.496483,3.700228,101.582376,421.517122,0.02574,0.347,1312.073149
1,2015,343.797,4.568463,3.804783,91.794692,434.828012,0.02574,0.347,1353.506487
2,2016,299.571,4.563431,3.905060,96.318143,395.230772,0.02574,0.347,1230.250578
3,2017,310.006,4.581153,3.991510,99.935946,409.352303,0.02574,0.347,1274.207231
4,2018,324.343,4.589904,4.068935,96.600049,420.422080,0.02574,0.347,1308.664567
5,2019,343.899,4.633224,4.137342,97.986091,441.389209,0.02574,0.347,1373.929786
6,2020,380.350,4.699734,4.202455,103.553749,483.406470,0.02574,0.347,1504.718589
7,2021,382.662,4.797094,4.267752,109.003947,491.136605,0.02574,0.347,1528.780487
8,2022,389.490,4.928803,4.337259,114.806528,503.704983,0.02574,0.347,1567.902580
9,2023,395.128,5.108207,4.414933,121.619274,516.054000,0.02574,0.347,1606.341858


## Section 199: Domestic production deduction

In [7]:
sec199_data = pd.read_csv(ctax_data_path + 'sec199.csv')
sec199_data
profit = sec199_data['profit'].tolist()
sec199 = sec199_data['sec199'][:9].tolist()
for i in range(9,23):
    sec199.append(sec199[i-1] * profit[i] / profit[i-1])
sec199_results = pd.DataFrame({'year': range(2014,2028), 'sec199': sec199[9:]})
## Note: In Tax-Calculator, this is e03240 and ALD_DOmesticProduction_hc

In [8]:
combined_base = combined_base.merge(right=sec199_results, how='outer', on='year')

## CCR Deduction model

In [10]:
## Extract dataset for MACRS model
## Run B-Tax to ensure baseline is set to 2017
ModelDiffs = run_btax(False, True, 2017)
btax_data = ModelDiffs[0]
btax_data.drop(['ADS Life', 'GDS Life', 'Asset Type', 'System', 'asset_category',
                'bea_asset_code', 'major_asset_group', 'metr_c', 'metr_c_d', 'metr_c_e', 'metr_nc', 'metr_nc_d',
                'metr_nc_e', 'mettr_c', 'mettr_c_d', 'mettr_c_e', 'mettr_nc', 'mettr_nc_d', 'mettr_nc_e', 'rho_c',
                'rho_c_d', 'rho_c_e', 'rho_nc', 'rho_nc_d', 'rho_nc_e', 'z_c', 'z_c_d', 'z_c_e', 'z_nc', 'z_nc_d',
                'z_nc_e', 'b'], axis=1, inplace=True)
btax_data['L'] = np.where(btax_data['GDS Class Life'] == 39.0, 39.5, btax_data['GDS Class Life'])
btax_data.drop([3, 21, 32, 91], axis=0, inplace=True)

econ_depr_filepath = 'btax_mini/data/depreciation_rates/Economic Depreciation Rates.csv'
df_econdepr = pd.read_csv(econ_depr_filepath)
df_econdepr['Asset'][78] = 'Communications equipment manufacturing'
df_econdepr['Asset'][81] = 'Motor vehicles and parts manufacturing'
df_econdepr.drop('Code', axis=1, inplace=True)
df_econdepr.rename(columns={'Economic Depreciation Rate': 'delta'}, inplace=True)
ccr_data = btax_data.merge(right=df_econdepr, how='outer', on='Asset')
ccr_data.drop([96, 97, 98], axis=0, inplace=True)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
{'tau_scg': 0.3200666497871944, 'tau_lcg': 0.22534310842801036, 'tau_h': 0.17128237349597358, 'tau_nc': 0.31782130579911383, 'tau_int': 0.34669502886889475, 'tau_div': 0.19140034273382414, 'tau_td': 0.24556278898243944}


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
base_data = copy.deepcopy(ccr_data)
investmentrate_data = pd.read_csv(ctax_data_path + 'investmentrates.csv')
investmentshare_data = pd.read_csv(ctax_data_path + 'investmentshares.csv')
investmentGfactors_data = pd.read_csv(ctax_data_path + 'investment_gfactors.csv')
depreciationIRS_data = pd.read_csv(ctax_data_path + 'dep data.csv')
bonus_data = pd.read_csv(ctax_data_path + 'bonus_data.csv')

In [12]:
def depreciationDeduction(year_investment, year_deduction, method, L, delta, bonusdata):
    # year_investment: year the investment was made
    # Method: Method of CCR (DB 200%, DB 150%, SL, Expensing)
    # L: class life
    # year_deduction: year of deduction calculation
    assert method in ['DB 200%', 'DB 150%', 'SL', 'Expensing', 'Economic']
    ##Extract bonus depreciation amount
    if L < 100:
        b = bonusdata['bonus' + str(int(L))][year_investment]
    else:
        b = 0
    if method == 'Expensing':
        if year_deduction == year_investment:
            deduction = 1.0
        else:
            deduction = 0
    elif method == 'Economic':
        pi_temp = investmentGfactors_data['pce'][year_deduction + 1] / investmentGfactors_data['pce'][year_deduction]
        if pi_temp == np.exp(delta):
            annual_change = 1.0
        else:
            annual_change = (pi_temp * np.exp(delta) - 1) / (np.log(pi_temp) - delta)
        if year_deduction < year_investment:
            deduction = 0
        elif year_deduction == year_investment:
            deduction = b + (1 - b) * delta * annual_change
        else:
            deduction = (investmentGfactors_data['pce'][year_deduction] / investmentGfactors_data['pce'][year_investment] *
                         np.exp(-delta * (year_deduction - year_investment)) * delta * annual_change) * (1 - b)
    else:
        if method == 'DB 200%':
            N = 2
        elif method == 'DB 150%':
            N = 1.5
        elif method == 'SL':
            N = 1
    ##Apply DDB and SL scenario
        t0 = year_investment + 0.5
        t1 = t0 + L * (1 - 1 / N)
        s1 = year_deduction
        s2 = s1 + 1
        if year_deduction < year_investment:
            deduction = 0
        elif year_deduction > year_investment + L:
            deduction = 0
        elif year_deduction == year_investment:
            deduction = b + (1 - b) * (1 - np.exp(-N / L * 0.5))
        elif s2 <= t1:
            deduction = (1 - b) * (np.exp(-N / L * (s1 - t0)) - np.exp(-N / L * (s2 - t0)))
        elif s1 >= t1 and s1 <= t0 + L and s2 > t0 + L:
            deduction = (1 - b) * (N / L * np.exp(1 - N) * (s2 - s1) * 0.5)
        elif s1 >= t1 and s2 <= t0 + L:
            deduction = (1 - b) * (N / L * np.exp(1 - N) * (s2 - s1))
        elif s1 < t1 and s2 > t1:
            deduction = (1 - b) * (np.exp(-N / L * (s1 - t0)) - np.exp(-N / L * (t1 - t0)) + N / L * np.exp(1 - N) * (s2 - t1))
        if L == 100:
            deduction = 0
    return(deduction)

In [13]:
def build_inv_matrix(corp_noncorp=True):
    inv_mat1 = np.zeros((96,75))
    # Adjust historical investment to just corporate
    for j in range(57):
        if corp_noncorp:
            inv_mat1[:,j] = investmentrate_data['i' + str(j + 1960)] * investmentshare_data['c_share'][j]
        else:
            inv_mat1[:,j] = investmentrate_data['i' + str(j + 1960)] * (1 - investmentshare_data['c_share'][j])
    # Extend investment using NGDP (growth factors from CBO forecast)
    for j in range(57,75):
        inv_mat1[:,j] = inv_mat1[:,56] * investmentGfactors_data['ngdp'][j] / investmentGfactors_data['ngdp'][56]
    # Rescale investment to match investment based on B-Tax for 2017
    if corp_noncorp:
        inv2017 = np.asarray(base_data['assets_c'] * (investmentGfactors_data['ngdp'][57] / investmentGfactors_data['ngdp'][56] - 1 + base_data['delta']))
    else:
        inv2017 = np.asarray(base_data['assets_nc'] * (investmentGfactors_data['ngdp'][57] / investmentGfactors_data['ngdp'][56] - 1 + base_data['delta']))
    inv_mat2 = np.zeros((96,75))
    l1 = range(96)
    l1.remove(32)
    for j in range(75):
        for i in l1:
            inv_mat2[i,j] = inv_mat1[i,j] * inv2017[i] / inv_mat1[i, 57]
    return(inv_mat2)
inv_mat_base_corp = build_inv_matrix()
inv_mat_base_noncorp = build_inv_matrix(corp_noncorp = False)

In [14]:
# Depreciation deduction 3d array
def calcDepAdjustment(corp_noncorp=True):
    if corp_noncorp:
        investment_matrix = build_inv_matrix()
    else:
        investment_matrix = build_inv_matrix(False)
    Depreciation_arr = np.zeros((96,75,75))
    for i in range(96): #asset
        for j in range(75): #year investment made
            for k in range(75): #year deduction taken
                Depreciation_arr[i,j,k] = depreciationDeduction(j, k, np.asarray(base_data['Method'])[i], 
                                                                np.asarray(base_data['L'])[i], 
                                                                np.asarray(base_data['delta'])[i], 
                                                                bonus_data) * investment_matrix[i,j]
    totalAnnualDepreciation = np.zeros(75)
    for k in range(75):
        totalAnnualDepreciation[k] = Depreciation_arr[:,:,k].sum().sum()
    depreciation_data = copy.deepcopy(depreciationIRS_data)
    depreciation_data['dep_model'] = totalAnnualDepreciation[40:54]
    if corp_noncorp:
        depreciation_data['scale'] = depreciation_data['dep_Ccorp'] / depreciation_data['dep_model']
    else:
        depreciation_data['scale'] = ((depreciation_data['dep_Scorp'] + depreciation_data['dep_sp'] + depreciation_data['dep_partner']) /
                                      depreciation_data['dep_model'])
    adj_factor = sum(depreciation_data['scale']) / len(depreciation_data['scale'])
    return(adj_factor)
adjfactor_dep_corp = calcDepAdjustment()
adjfactor_dep_noncorp = calcDepAdjustment(False)

In [15]:
def annualCCRdeduction(investment_matrix, bonusdata, adj_factor, hc_undep=0., hc_undep_year=0):
    # investment_matrix: the matrix (asset type x year investment made) of investment
    # hc_undep, hc_undep_year: haircut on depreciation deductions taken after hc_under_year on investments made before hc_undep_year
    Depreciation_3darray = np.zeros((96,75,75))
    for i in range(96):
        for j in range(75):
            for k in range(75):
                Depreciation_3darray[i,j,k] = (depreciationDeduction(j, k, np.asarray(base_data['Method'])[i],
                                                                     np.asarray(base_data['L'])[i], 
                                                                     np.asarray(base_data['delta'])[i], bonusdata) * investment_matrix[i,j])
    for j in range(75):
        for k in range(75):
            if j + 1960 < hc_undep_year and k + 1960 >= hc_undep_year:
                Depreciation_3darray[:,j,k] = Depreciation_3darray[:,j,k] * (1 - hc_undep)
    totalAnnualDeduction = np.zeros(75)
    for k in range(75):
        totalAnnualDeduction[k] = Depreciation_3darray[:,:,k].sum().sum() * adjfactor_dep_corp
    return totalAnnualDeduction
annualDepreciation_base_corp = annualCCRdeduction(inv_mat_base_corp, bonus_data, adjfactor_dep_corp)
annualDepreciation_base_noncorp = annualCCRdeduction(inv_mat_base_noncorp, bonus_data, adjfactor_dep_noncorp)

In [16]:
def capitalPath(investment_mat, depDeduction_vec, adj_factor, corp_noncorp=True, economic=False):
    Kstock = np.zeros((96,15))
    trueDep = np.zeros((96,14))
    for i in range(96):
        if corp_noncorp:
            Kstock[i,3] = np.asarray(base_data['assets_c'])[i]
        else:
            Kstock[i,3] = np.asarray(base_data['assets_nc'])[i]
        for j in [56,55,54]:
            Kstock[i,j-54] = (Kstock[i,j-53] * investmentGfactors_data['pce'][j] / 
                              investmentGfactors_data['pce'][j+1] - investment_mat[i,j]) / (1 - np.asarray(base_data['delta'])[i])
            trueDep[i,j-54] = Kstock[i,j-54] * np.asarray(base_data['delta'])[i]
        for j in range(57,68):
            trueDep[i,j-54] = Kstock[i,j-54] * np.asarray(base_data['delta'])[i]
            Kstock[i,j-53] = ((Kstock[i,j-54] + investment_mat[i,j] - trueDep[i,j-54]) * 
                              investmentGfactors_data['pce'][j+1] / investmentGfactors_data['pce'][j])
    # Sum across assets and put into new dataset
    Kstock_total = np.zeros(14)
    fixedK_total = np.zeros(14)
    trueDep_total = np.zeros(14)
    inv_total = np.zeros(14)
    fixedInv_total = np.zeros(14)
    Mdep_total = np.zeros(14)
    for j in range(14):
        Kstock_total[j] = sum(Kstock[:,j]) * adj_factor
        fixedK_total[j] = (sum(Kstock[:,j]) - Kstock[31,j] - Kstock[32,j]) * adj_factor
        trueDep_total[j] = sum(trueDep[:,j]) * adj_factor
        inv_total[j] = sum(investment_mat[:,j+54]) * adj_factor
        fixedInv_total[j] = (sum(investment_mat[:,j+54]) - investment_mat[31,j+54]) * adj_factor
        Mdep_total[j] = depDeduction_vec[j+54]
    capital_path_results = pd.DataFrame({'year': range(2014,2028), 'Kstock': Kstock_total, 'Investment': inv_total,
                                        'FixedInv': fixedInv_total, 'TrueDep': trueDep_total, 'taxDep': Mdep_total,
                                        'FixedK': fixedK_total})
    if economic:
        taxDep_results = capital_path_results.drop(['Kstock', 'FixedK', 'Investment', 'FixedInv', 'taxDep'], axis=1)
        taxDep_results.rename(columns={'TrueDep': 'taxDep'}, inplace=True)
    else:
        taxDep_results = capital_path_results.drop(['Kstock', 'FixedK', 'Investment', 'FixedInv', 'TrueDep'], axis=1)
    return (capital_path_results, taxDep_results)

(capPath_base_corp, taxDep_base_corp) = capitalPath(inv_mat_base_corp, annualDepreciation_base_corp, adjfactor_dep_corp)
(capPath_base_noncorp, taxDep_base_noncorp) = capitalPath(inv_mat_base_noncorp, annualDepreciation_base_noncorp, adjfactor_dep_noncorp, False)

In [17]:
combined_base = combined_base.merge(right=taxDep_base_corp, how='outer', on='year')

In [18]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1094.081762,6591.805053,1177.556186,9073.744596,508.231237,902.336891,2014
1,1048.886439,7198.960307,1163.657496,9771.988941,581.624855,890.312305,2015
2,944.754393,7757.812380,974.598908,10477.723962,639.197568,827.819233,2016
3,979.542579,8201.071455,1010.486042,10997.786532,677.558912,852.830903,2017
4,1020.509244,8667.183456,1052.746832,11549.422248,716.068312,807.392625,2018
5,1057.473593,9150.500719,1090.878874,12123.085955,755.035167,787.094657,2019
6,1093.940824,9644.505887,1128.498094,12711.413734,793.791020,633.603895,2020
7,1135.145903,10147.176540,1171.004829,13311.802351,832.331229,714.119119,2021
8,1180.008357,10660.098127,1217.284475,13924.931622,871.245007,785.425745,2022
9,1226.864361,11188.316296,1265.620646,14556.491460,911.182627,846.027716,2023


In [19]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,319.682732,3024.991042,360.948991,7936.559218,187.766689,283.773872,2014
1,338.357767,3166.277423,398.286254,8133.812870,200.564175,293.175099,2015
2,344.877587,3343.545672,361.548895,8431.074117,214.660258,293.423038,2016
3,357.576830,3533.086215,374.862016,8724.452978,227.479886,306.150800,2017
4,372.531494,3733.890939,390.539586,9043.081737,240.408819,296.495393,2018
5,386.025133,3943.094228,404.685505,9376.506860,253.528055,293.986117,2019
6,399.337303,4158.184428,418.641182,9720.745381,266.620942,249.537756,2020
7,414.378999,4378.284093,434.409990,10073.822444,279.682351,279.798753,2021
8,430.755800,4603.718468,451.578442,10434.204714,292.894337,306.177545,2022
9,447.860336,4836.445070,469.509808,10804.821526,306.466520,328.821334,2023


## NID model

In [20]:
## Nonfinancial corporate sector
debt_data_corp = pd.read_csv(ctax_data_path + 'Corp debt data.csv')

def calcNIDscale(capital_path, eta=0.4):
    # capital_path: growth path of the capital stock
    # eta: retirement rate of existing debt
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_corp['Kfa'][:57].tolist()
    A = debt_data_corp['A'][:57].tolist()
    L = debt_data_corp['L'][:57].tolist()
    D = [L[i] - A[i] for i in range(len(L))]
    i_t = debt_data_corp['i_t'].tolist()
    i_pr = debt_data_corp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        A.append(A[56] * K_fa[i] / K_fa[56])
        D.append(D[56] * K_fa[i] / K_fa[56])
        L.append(D[i] - A[i])
    R = np.zeros(68)
    O = np.zeros(68)
    for i in range(1,68):
        R[i] = L[i-1] * eta
        O[i] = L[i] - L[i-1] + R[i]
    i_a = [x / 100. for x in i_t]
    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_income = [A[i] * i_a[i] for i in range(len(A))]
    int_expense = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j]
    NID_gross = int_expense - int_income
    NID_scale = sum((debt_data_corp['NID_IRS'][38:54] / NID_gross[38:54])) / 16.
    return NID_scale
adjfactor_nid_corp = calcNIDscale(capPath_base_corp)


def netInterestDeduction(capital_path, eta=0.4, id_hc_year=0, nid_hc_year=0, id_hc_old=0, id_hc_new=0, nid_hc_old=0, nid_hc=0):
    # capital_path: growth path of the capital stock
    # eta: retirement rate of existing debt
    # nid_hc: haircut on the net interest deduction, beginning in nid_hc_year
    # id_hc_old, id_hc_new: haircuts on the deduction of interest paid on debt originated before id_hc_year and on debt originated beginning in id_hc_year
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_corp['Kfa'][:57].tolist()
    A = debt_data_corp['A'][:57].tolist()
    L = debt_data_corp['L'][:57].tolist()
    D = [L[i] - A[i] for i in range(len(L))]
    i_t = debt_data_corp['i_t'].tolist()
    i_pr = debt_data_corp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        A.append(A[56] * K_fa[i] / K_fa[56])
        D.append(D[56] * K_fa[i] / K_fa[56])
        L.append(D[i] + A[i])
    R = np.zeros(68)
    O = np.zeros(68)
    for i in range(1,68):
        R[i] = L[i-1] * eta
        O[i] = L[i] - L[i-1] + R[i]
    i_a = [x / 100. for x in i_t]
    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_income = [A[i] * i_a[i] for i in range(len(A))]
    int_expense = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            if j + 1960 < id_hc_year and i + 1960 >= id_hc_year:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_old)
            elif j + 1960 >= id_hc_year:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_new)
            else:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j]
    NID_gross = int_expense - int_income
    NID = np.zeros(len(NID_gross))
    NIP = NID_gross * adjfactor_nid_corp
    for i in range(len(NID)):
        if i + 1960 < nid_hc_year:
            NID[i] = NID_gross[i] * adjfactor_nid_corp
        else:
            NID[i] = NID_gross[i] * adjfactor_nid_corp * (1 - nid_hc)
    NID_results = pd.DataFrame({'year': range(2014,2028), 'nid': NID[54:68]})
    NIP_results = pd.DataFrame({'year': range(2014,2028), 'nip': NIP[54:68]})
    #NID_results = pd.DataFrame({'year': range(1998,2014), 'nid': NID[38:54]}) #for printing historical results
    return (NID_results, NIP_results)
(NID_base, NIP_base) = netInterestDeduction(capPath_base_corp)

In [21]:
## Nonfinancial noncorporate sector

debt_data_noncorp = pd.read_csv(ctax_data_path + 'Noncorp debt data.csv')

def calcIDscale_noncorp(capital_path, eta=0.4):
    # capital_path: growth path of the capital stock
    # eta: retirement rate of existing debt
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_noncorp['Kfa'][:57].tolist()
    L = debt_data_noncorp['L'][:57].tolist()
    i_t = debt_data_noncorp['i_t'].tolist()
    i_pr = debt_data_noncorp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        L.append(L[56] * K_fa[i] / K_fa[56])
    R = np.zeros(68)
    O = np.zeros(68)
    for i in range(1,68):
        R[i] = L[i-1] * eta
        O[i] = L[i] - L[i-1] + R[i]
    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_expense = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j]
    ID_scale = sum(((debt_data_noncorp['ID_Scorp'][38:54] + debt_data_noncorp['ID_sp'][38:54] + 
                     debt_data_noncorp['ID_partner'][38:54]) / int_expense[38:54])) / 16.
    return ID_scale
adjfactor_id_noncorp = calcIDscale_noncorp(capPath_base_noncorp)

def noncorpIntDeduction(capital_path, eta=0.4):
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_noncorp['Kfa'][:57].tolist()
    L = debt_data_noncorp['L'][:57].tolist()
    i_t = debt_data_noncorp['i_t'].tolist()
    i_pr = debt_data_noncorp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        L.append(L[56] * K_fa[i] / K_fa[56])
    R = np.zeros(68)
    O = np.zeros(68)
    for i in range(1,68):
        R[i] = L[i-1] * eta
        O[i] = L[i] - L[i-1] + R[i]
    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_expense = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j]
    int_total = int_expense * adjfactor_id_noncorp
    ID_results = pd.DataFrame({'year': range(2014,2028), 'intpaid': int_total[54:68]})
    #ID_results = pd.DataFrame({'year': range(1998,2014), 'intpaid': int_total[38:54]})
    return ID_results
IntPaid_base_noncorp = noncorpIntDeduction(capPath_base_noncorp)

In [22]:
combined_base = combined_base.merge(right=NID_base, how='outer', on='year')
combined_base['ebitda'] = combined_base['taxinc'] + combined_base['sec199'] + combined_base['taxDep'] + combined_base['nid']
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.496483,3.700228,101.582376,421.517122,0.02574,0.347,1312.073149,34.306208,902.336891,231.441998,2480.158247
1,2015,343.797,4.568463,3.804783,91.794692,434.828012,0.02574,0.347,1353.506487,34.855388,890.312305,243.981131,2522.655310
2,2016,299.571,4.563431,3.905060,96.318143,395.230772,0.02574,0.347,1230.250578,34.816996,827.819233,248.001094,2340.887900
3,2017,310.006,4.581153,3.991510,99.935946,409.352303,0.02574,0.347,1274.207231,34.952204,852.830903,268.262930,2430.253269
4,2018,324.343,4.589904,4.068935,96.600049,420.422080,0.02574,0.347,1308.664567,35.018974,807.392625,295.580925,2446.657090
5,2019,343.899,4.633224,4.137342,97.986091,441.389209,0.02574,0.347,1373.929786,35.349483,787.094657,328.934835,2525.308762
6,2020,380.350,4.699734,4.202455,103.553749,483.406470,0.02574,0.347,1504.718589,35.856932,633.603895,362.449185,2536.628601
7,2021,382.662,4.797094,4.267752,109.003947,491.136605,0.02574,0.347,1528.780487,36.599743,714.119119,393.699349,2673.198699
8,2022,389.490,4.928803,4.337259,114.806528,503.704983,0.02574,0.347,1567.902580,37.604625,785.425745,422.521900,2813.454850
9,2023,395.128,5.108207,4.414933,121.619274,516.054000,0.02574,0.347,1606.341858,38.973402,846.027716,448.489428,2939.832404


# Apply partial equilibrium responses

## Specify economic and tax parameters

In [23]:
## Economic assumptions
p = 0.2     #real financial return
f = 0.32    #debt financing share
i_d = 0.068   #nominal interest rate on debt
pi = 0.024  #inflation rate
E = 0.058   #real return on equity

## Tax policy parameters
reform_start_year = 2017
tau_base = 0.35     #statutory tax rate, current law
tau_ref = 0.36      #statutory tax rate
progressive_rate = True
int_hc = 0.0        #haircut on interest deduction
## Modify bonus depreciation dataset to use new laws
bonus_data_ref = copy.deepcopy(bonus_data)
#for i in range(len(bonus_data_ref)):
#    if bonus_data_ref['year'][i] >= reform_start_year:
#        bonus_data_ref['bonus3'][i] = 1.
#        bonus_data_ref['bonus5'][i] = 1.
#        bonus_data_ref['bonus7'][i] = 1.
#        bonus_data_ref['bonus10'][i] = 1.
#        bonus_data_ref['bonus15'][i] = 1.
#        bonus_data_ref['bonus20'][i] = 1.
#        bonus_data_ref['bonus25'][i] = 1.
#        bonus_data_ref['bonus27'][i] = 1.
#        bonus_data_ref['bonus39'][i] = 1.

inventory_accounting = 0 #0 for historical mix of FIFO and LIFO, 1 for FIFO, 2 for LIFO, 3 for expensing
bonusrate_land = 0 #bonus depreciation rate for land. Does not affect model results

## Calculate the investment response

In [24]:
def investmentResponse(startyear, elast_usercost_c=0, elast_usercost_nc=0, elast_eatr=0):
    assert startyear >= 2017
    assert elast_usercost_c <= 0
    assert elast_usercost_nc <= 0
    assert elast_eatr <= 0
    combined_response = copy.deepcopy(ccr_data)
    combined_response.drop(['GDS Class Life', 'assets_c', 'assets_nc', 'Method', 'bonus', 'L', 'delta'], axis=1, inplace=True)
    for year in range(2014, 2028):
        if year < startyear:
            combined_response['deltaIc' + str(year)] = 0
            combined_response['deltaEc' + str(year)] = 0
            combined_response['deltaInc' + str(year)] = 0
            combined_response['deltaEnc' + str(year)] = 0
        elif (elast_usercost_c == 0) and (elast_usercost_nc == 0) and (elast_eatr == 0):
            combined_response['deltaIc' + str(year)] = 0
            combined_response['deltaEc' + str(year)] = 0
            combined_response['deltaInc' + str(year)] = 0
            combined_response['deltaEnc' + str(year)] = 0
        else:
            ## Get results for the year
            BtaxBase = run_btax(False, year, {}, btax_betr_corp=tau_base, btax_betr_entity_Switch=False, btax_betr_pass=0,
                                btax_depr_3yr_exp=bonus_data['bonus3'][year-1960] * 100.,
                                btax_depr_5yr_exp=bonus_data['bonus5'][year-1960] * 100.,
                                btax_depr_7yr_exp=bonus_data['bonus7'][year-1960] * 100.,
                                btax_depr_10yr_exp=bonus_data['bonus10'][year-1960] * 100.,
                                btax_depr_15yr_exp=bonus_data['bonus15'][year-1960] * 100.,
                                btax_depr_20yr_exp=bonus_data['bonus20'][year-1960] * 100.,
                                btax_depr_25yr_exp=bonus_data['bonus25'][year-1960] * 100.,
                                btax_depr_27_5yr_exp=bonus_data['bonus27'][year-1960] * 100.,
                                btax_depr_39yr_exp=bonus_data['bonus39'][year-1960] * 100.,
                                inventory_method=inventory_accounting, btax_depr_land_exp=bonusrate_land,
                                btax_other_hair=0, btax_other_corpeq=0, btax_other_invest=0, btax_other_proptx=0,
                                btax_depr_allyr_ads_Switch=False, btax_depr_allyr_exp=0, btax_depr_allyr_gds_Switch=True, btax_depr_allyr_tax_Switch=False,
                                btax_depr_hover_ads_Switch=False, btax_depr_hover_exp=0, btax_depr_hover_gds_Switch=True, btax_depr_hover_tax_Switch=False,
                                btax_depr_3yr_ads_Switch=False, btax_depr_3yr_gds_Switch=True, btax_depr_3yr_tax_Switch=False,
                                btax_depr_5yr_ads_Switch=False, btax_depr_5yr_gds_Switch=True, btax_depr_5yr_tax_Switch=False,
                                btax_depr_7yr_ads_Switch=False, btax_depr_7yr_gds_Switch=True, btax_depr_7yr_tax_Switch=False,
                                btax_depr_10yr_ads_Switch=False, btax_depr_10yr_gds_Switch=True, btax_depr_10yr_tax_Switch=False,
                                btax_depr_15yr_ads_Switch=False, btax_depr_15yr_gds_Switch=True, btax_depr_15yr_tax_Switch=False,
                                btax_depr_20yr_ads_Switch=False, btax_depr_20yr_gds_Switch=True, btax_depr_20yr_tax_Switch=False,
                                btax_depr_25yr_ads_Switch=False, btax_depr_25yr_gds_Switch=True, btax_depr_25yr_tax_Switch=False,
                                btax_depr_27_5yr_ads_Switch=False, btax_depr_27_5yr_gds_Switch=True, btax_depr_27_5yr_tax_Switch=False,
                                btax_depr_39yr_ads_Switch=False, btax_depr_39yr_gds_Switch=True, btax_depr_39yr_tax_Switch=False)
            BtaxRefm = run_btax(False, year, {}, btax_betr_corp=tau_ref, btax_betr_entity_Switch=False, btax_betr_pass=0,
                                btax_depr_3yr_exp=bonus_data_ref['bonus3'][year-1960] * 100.,
                                btax_depr_5yr_exp=bonus_data_ref['bonus5'][year-1960] * 100.,
                                btax_depr_7yr_exp=bonus_data_ref['bonus7'][year-1960] * 100.,
                                btax_depr_10yr_exp=bonus_data_ref['bonus10'][year-1960] * 100.,
                                btax_depr_15yr_exp=bonus_data_ref['bonus15'][year-1960] * 100.,
                                btax_depr_20yr_exp=bonus_data_ref['bonus20'][year-1960] * 100.,
                                btax_depr_25yr_exp=bonus_data_ref['bonus25'][year-1960] * 100.,
                                btax_depr_27_5yr_exp=bonus_data_ref['bonus27'][year-1960] * 100.,
                                btax_depr_39yr_exp=bonus_data_ref['bonus39'][year-1960] * 100.,
                                inventory_method=inventory_accounting, btax_depr_land_exp=bonusrate_land,
                                btax_other_hair=int_hc, btax_other_corpeq=0, btax_other_invest=0, btax_other_proptx=0,
                                btax_depr_allyr_ads_Switch=False, btax_depr_allyr_exp=0, btax_depr_allyr_gds_Switch=True, btax_depr_allyr_tax_Switch=False,
                                btax_depr_hover_ads_Switch=False, btax_depr_hover_exp=0, btax_depr_hover_gds_Switch=True, btax_depr_hover_tax_Switch=False,
                                btax_depr_3yr_ads_Switch=False, btax_depr_3yr_gds_Switch=True, btax_depr_3yr_tax_Switch=False,
                                btax_depr_5yr_ads_Switch=False, btax_depr_5yr_gds_Switch=True, btax_depr_5yr_tax_Switch=False,
                                btax_depr_7yr_ads_Switch=False, btax_depr_7yr_gds_Switch=True, btax_depr_7yr_tax_Switch=False,
                                btax_depr_10yr_ads_Switch=False, btax_depr_10yr_gds_Switch=True, btax_depr_10yr_tax_Switch=False,
                                btax_depr_15yr_ads_Switch=False, btax_depr_15yr_gds_Switch=True, btax_depr_15yr_tax_Switch=False,
                                btax_depr_20yr_ads_Switch=False, btax_depr_20yr_gds_Switch=True, btax_depr_20yr_tax_Switch=False,
                                btax_depr_25yr_ads_Switch=False, btax_depr_25yr_gds_Switch=True, btax_depr_25yr_tax_Switch=False,
                                btax_depr_27_5yr_ads_Switch=False, btax_depr_27_5yr_gds_Switch=True, btax_depr_27_5yr_tax_Switch=False,
                                btax_depr_39yr_ads_Switch=False, btax_depr_39yr_gds_Switch=True, btax_depr_39yr_tax_Switch=False)
            df_base = BTaxBase[0]
            df_ref = BTaxRefm[0]
            ## Build main dataset
            df_base.drop(['ADS Life', 'Asset Type', 'GDS Class Life', 'GDS Life', 'Method', 'System', 'asset_category',
                          'b', 'bea_asset_code', 'bonus', 'major_asset_group', 'metr_c', 'metr_c_d', 'metr_c_e',
                          'metr_nc', 'metr_nc_d', 'metr_nc_e', 'mettr_c', 'mettr_c_d', 'mettr_c_e', 'mettr_nc', 'mettr_nc_d',
                          'mettr_nc_e', 'rho_c_d', 'rho_c_e', 'rho_nc_d', 'rho_nc_e', 'z_c_d', 'z_c_e',
                          'z_nc_d', 'z_nc_e'], axis=1, inplace=True)
            df_ref.drop(['ADS Life', 'Asset Type', 'GDS Class Life', 'GDS Life', 'Method', 'System', 'asset_category', 
                         'b', 'bea_asset_code', 'bonus', 'major_asset_group', 'metr_c', 'metr_c_d', 'metr_c_e',
                         'metr_nc', 'metr_nc_d', 'metr_nc_e', 'mettr_c', 'mettr_c_d', 'mettr_c_e', 'mettr_nc', 'mettr_nc_d',
                         'mettr_nc_e', 'rho_c_d', 'rho_c_e', 'rho_nc_d', 'rho_nc_e', 'z_c_d', 'z_c_e', 
                         'z_nc_d', 'z_nc_e', 'assets_c', 'assets_nc'], axis=1, inplace=True)
            econ_depr_filepath = "btax/data/depreciation_rates/Economic Depreciation Rates.csv"
            df_econdepr = pd.read_csv(econ_depr_filepath)
            df_econdepr['Asset'][78] = 'Communications equipment manufacturing'
            df_econdepr['Asset'][81] = 'Motor vehicles and parts manufacturing'
            df_econdepr.drop('Code', axis=1, inplace=True)
            df_base.rename(columns={'assets_c': 'K_c_base', 'rho_c': 'rho_c_base', 'z_c': 'z_c_base', 
                                    'assets_nc': 'K_nc_base', 'rho_nc': 'rho_nc_base', 'z_nc': 'z_nc_base'}, inplace=True)
            df_ref.rename(columns={'rho_c': 'rho_c_ref', 'z_c': 'z_c_ref', 'rho_nc': 'rho_nc_ref', 'z_nc': 'z_nc_ref'}, inplace=True)
            df_econdepr.rename(columns={'Economic Depreciation Rate': 'delta'}, inplace=True)
            df_comb = df_base.merge(right=df_ref, how='outer', on='Asset')
            df_total = df_comb.merge(right=df_econdepr, how='outer', on='Asset')
            df_nocat = df_total.drop([3, 21, 32, 91, 100, 101, 102], axis=0)
            
            ## Apply marginal response using user cost of capital, to corporate and noncorporate
            df_nocat['usercost_c_base'] = df_nocat['rho_c_base'] + df_nocat['delta']
            df_nocat['usercost_c_ref'] = df_nocat['rho_c_ref'] + df_nocat['delta']
            df_nocat['deltaInv_c_m'] = ((df_nocat['usercost_c_ref'] / df_nocat['usercost_c_base']) - 1) * elast_usercost_c
            df_nocat['deltaE_c_m'] = (df_nocat['usercost_c_base'] + df_nocat['usercost_c_ref']) / 2.0
            df_nocat['usercost_nc_base'] = df_nocat['rho_nc_base'] + df_nocat['delta']
            df_nocat['usercost_nc_ref'] = df_nocat['rho_nc_ref'] + df_nocat['delta']
            df_nocat['deltaInv_nc'] = ((df_nocat['usercost_nc_ref'] / df_nocat['usercost_nc_base']) - 1) * elast_usercost_nc
            df_nocat['deltaE_nc'] = (df_nocat['usercost_nc_base'] + df_nocat['usercost_nc_ref']) / 2.0
            
            ## Apply nonmarginal response using EATR, to corporate only
            r = (1 - f) * E + f * (i_d - pi)
            F_base = f * tau_base
            F_ref = f * tau_ref * (1 - int_hc)

            df_nocat['Rstar'] = (p - r) / (r + df_nocat['delta'])
            df_nocat['P'] = p / (r + df_nocat['delta'])
            df_nocat['R_base'] = (-(1 - df_nocat['z_c_base'] * tau_base - F_base) +
                                  (p + df_nocat['delta']) * (1 - tau_base) / (r + df_nocat['delta']))
            df_nocat['R_ref'] = (-(1 - df_nocat['z_c_ref'] * tau_ref - F_ref) +
                                 (p + df_nocat['delta']) * (1 - tau_ref) / (r + df_nocat['delta']))
            df_nocat['EATR_base'] = (df_nocat['Rstar'] - df_nocat['R_base']) / df_nocat['P']
            df_nocat['EATR_ref'] = (df_nocat['Rstar'] - df_nocat['R_ref']) / df_nocat['P']
            df_nocat['deltaInv_c_nm'] = (df_nocat['EATR_ref'] - df_nocat['EATR_base']) * elast_eatr
            df_nocat['deltaE_c_nm'] = p
            df_nocat['deltaInv_c'] = df_nocat['deltaInv_c_m'] + df_nocat['deltaInv_c_nm']
            df_nocat['deltaE_c'] = ((df_nocat['deltaE_c_m'] * df_nocat['deltaInv_c_m'] + 
                                     df_nocat['deltaE_c_nm'] * df_nocat['deltaInv_c_nm']) / (df_nocat['deltaInv_c'] + 0.0001))
            df_deltaI = df_nocat.drop(['K_c_base', 'rho_c_base', 'z_c_base', 'rho_c_ref', 'z_c_ref', 
                                       'K_nc_base', 'rho_nc_base', 'z_nc_base', 'rho_nc_ref', 'z_nc_ref',
                                       'delta', 'usercost_c_base', 'usercost_c_ref', 'usercost_nc_base', 'usercost_nc_ref',
                                       'deltaInv_c_m', 'Rstar', 'P', 'R_base', 'R_ref', 'EATR_base', 'EATR_ref', 'deltaInv_c_nm',
                                       'deltaE_c_m', 'deltaE_c_nm'], axis=1)
            df_deltaI.rename(columns={'deltaInv_c': 'deltaIc' + str(year), 'deltaE_c': 'deltaEc' + str(year),
                                      'deltaInv_nc': 'deltaInc' + str(year), 'deltaE_nc': 'deltaEnc' + str(year)}, inplace=True)
            combined_response = combined_response.merge(right=df_deltaI, how='outer', on='Asset')
    for year in range(2021,2028):
        combined_response['deltaIc' + str(year)] = combined_response['deltaIc2020']
        combined_response['deltaEc' + str(year)] = combined_response['deltaEc2020']
        combined_response['deltaInc' + str(year)] = combined_response['deltaInc2020']
        combined_response['deltaEnc' + str(year)] = combined_response['deltaEnc2020']
    return combined_response

In [25]:
response_results = investmentResponse(reform_start_year, elast_usercost_c=0.0, elast_usercost_nc=0, elast_eatr=0)

In [26]:
response_results

,Asset,deltaIc2014,deltaEc2014,deltaInc2014,deltaEnc2014,deltaIc2015,deltaEc2015,deltaInc2015,deltaEnc2015,deltaIc2016,...,deltaInc2025,deltaEnc2025,deltaIc2026,deltaEc2026,deltaInc2026,deltaEnc2026,deltaIc2027,deltaEc2027,deltaInc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Air transportation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aircraft,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"All other nonmanufacturing, n.e.c.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amusement and recreation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Autos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Books,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Chemical manufacturing, ex. pharma and med",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Communication,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Communications,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
## Create new investment matrices for corporate and noncorporate
def buildNewInvMatrix(response_data, response_type='usercost'):
    assert response_type in ['usercost', 'eatr']
    if response_type == 'usercost':
        invbase = 'gross'
    else:
        invbase = 'net'
    inv_mat_base_corp = build_inv_matrix()
    inv_mat_base_noncorp = build_inv_matrix(False)
    inv_mat_ref_corp = build_inv_matrix()
    inv_mat_ref_noncorp = build_inv_matrix(False)
    data_main = copy.deepcopy(ccr_data)
    data_main.drop(['GDS Class Life', 'Method', 'bonus', 'L'], axis=1, inplace=True)
    data_main.rename(columns={'assets_c': 'K_c_2017', 'assets_nc': 'K_nc_2017'}, inplace=True)
    for i in range(57,68):
        data_main['K_c_' + str(1961+i)] = ((data_main['K_c_' + str(1960+i)] * (1 - data_main['delta']) + 
                                            inv_mat_base_corp[:,i]) * investmentGfactors_data['pce'][i+1] / 
                                           investmentGfactors_data['pce'][i])
        data_main['Inv_c_' + str(1960+i)] = inv_mat_base_corp[:,i]
        data_main['NetInv_c_' + str(1960+i)] = data_main['Inv_c_' + str(1960+i)] - data_main['K_c_' + str(1960+i)] * data_main['delta']
        data_main['K_nc_' + str(1961+i)] = ((data_main['K_nc_' + str(1960+i)] * (1 - data_main['delta']) + 
                                             inv_mat_base_noncorp[:,i]) * investmentGfactors_data['pce'][i+1] / 
                                            investmentGfactors_data['pce'][i])
        data_main['Inv_nc_' + str(1960+i)] = inv_mat_base_noncorp[:,i]
        data_main['NetInv_nc_' + str(1960+i)] = data_main['Inv_nc_' + str(1960+i)] - data_main['K_nc_' + str(1960+i)] * data_main['delta']
        if invbase == 'gross':
            inv_mat_ref_corp[:,i] = inv_mat_base_corp[:,i] + data_main['Inv_c_' + str(1960+i)] * response_data['deltaIc' + str(1960+i)]
            inv_mat_ref_noncorp[:,i] = inv_mat_base_noncorp[:,i] + data_main['Inv_nc_' + str(1960+i)] * response_data['deltaInc' + str(1960+i)]
        else:
            inv_mat_ref_corp[:,i] = inv_mat_base_corp[:,i] + data_main['NetInv_c_' + str(1960+i)] * response_data['deltaIc' + str(1960+i)]
            inv_mat_ref_noncorp[:,i] = inv_mat_base_noncorp[:,i] + data_main['NetInv_nc_' + str(1960+i)] * response_data['deltaInc' + str(1960+i)]
    return (inv_mat_ref_corp, inv_mat_ref_noncorp)
(inv_mat_ref_corp, inv_mat_ref_noncorp) = buildNewInvMatrix(response_results, 'usercost')

In [28]:
## Redo capital calculations with new investment matrices
annualDepreciation_ref_corp = annualCCRdeduction(inv_mat_ref_corp, bonus_data_ref, adjfactor_dep_corp)
annualDepreciation_ref_noncorp = annualCCRdeduction(inv_mat_ref_noncorp, bonus_data_ref, adjfactor_dep_noncorp)

In [29]:
## Generate new capital stock path
(capPath_ref_corp, taxDep_ref_corp) = capitalPath(inv_mat_ref_corp, annualDepreciation_ref_corp, adjfactor_dep_corp)
(capPath_ref_noncorp, taxDep_ref_noncorp) = capitalPath(inv_mat_ref_noncorp, annualDepreciation_ref_noncorp, adjfactor_dep_noncorp, False)

In [30]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1094.081762,6591.805053,1177.556186,9073.744596,508.231237,902.336891,2014
1,1048.886439,7198.960307,1163.657496,9771.988941,581.624855,890.312305,2015
2,944.754393,7757.812380,974.598908,10477.723962,639.197568,827.819233,2016
3,979.542579,8201.071455,1010.486042,10997.786532,677.558912,852.830903,2017
4,1020.509244,8667.183456,1052.746832,11549.422248,716.068312,807.392625,2018
5,1057.473593,9150.500719,1090.878874,12123.085955,755.035167,787.094657,2019
6,1093.940824,9644.505887,1128.498094,12711.413734,793.791020,633.603895,2020
7,1135.145903,10147.176540,1171.004829,13311.802351,832.331229,714.119119,2021
8,1180.008357,10660.098127,1217.284475,13924.931622,871.245007,785.425745,2022
9,1226.864361,11188.316296,1265.620646,14556.491460,911.182627,846.027716,2023


In [31]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1094.081762,6591.805053,1177.556186,9073.744596,508.231237,902.336891,2014
1,1048.886439,7198.960307,1163.657496,9771.988941,581.624855,890.312305,2015
2,944.754393,7757.812380,974.598908,10477.723962,639.197568,827.819233,2016
3,979.542579,8201.071455,1010.486042,10997.786532,677.558912,852.830903,2017
4,1020.509244,8667.183456,1052.746832,11549.422248,716.068312,807.392625,2018
5,1057.473593,9150.500719,1090.878874,12123.085955,755.035167,787.094657,2019
6,1093.940824,9644.505887,1128.498094,12711.413734,793.791020,633.603895,2020
7,1135.145903,10147.176540,1171.004829,13311.802351,832.331229,714.119119,2021
8,1180.008357,10660.098127,1217.284475,13924.931622,871.245007,785.425745,2022
9,1226.864361,11188.316296,1265.620646,14556.491460,911.182627,846.027716,2023


In [32]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,319.682732,3024.991042,360.948991,7936.559218,187.766689,283.773872,2014
1,338.357767,3166.277423,398.286254,8133.812870,200.564175,293.175099,2015
2,344.877587,3343.545672,361.548895,8431.074117,214.660258,293.423038,2016
3,357.576830,3533.086215,374.862016,8724.452978,227.479886,306.150800,2017
4,372.531494,3733.890939,390.539586,9043.081737,240.408819,296.495393,2018
5,386.025133,3943.094228,404.685505,9376.506860,253.528055,293.986117,2019
6,399.337303,4158.184428,418.641182,9720.745381,266.620942,249.537756,2020
7,414.378999,4378.284093,434.409990,10073.822444,279.682351,279.798753,2021
8,430.755800,4603.718468,451.578442,10434.204714,292.894337,306.177545,2022
9,447.860336,4836.445070,469.509808,10804.821526,306.466520,328.821334,2023


In [33]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,319.682732,3024.991042,360.948991,7936.559218,187.766689,283.773872,2014
1,338.357767,3166.277423,398.286254,8133.812870,200.564175,293.175099,2015
2,344.877587,3343.545672,361.548895,8431.074117,214.660258,293.423038,2016
3,357.576830,3533.086215,374.862016,8724.452978,227.479886,306.150800,2017
4,372.531494,3733.890939,390.539586,9043.081737,240.408819,296.495393,2018
5,386.025133,3943.094228,404.685505,9376.506860,253.528055,293.986117,2019
6,399.337303,4158.184428,418.641182,9720.745381,266.620942,249.537756,2020
7,414.378999,4378.284093,434.409990,10073.822444,279.682351,279.798753,2021
8,430.755800,4603.718468,451.578442,10434.204714,292.894337,306.177545,2022
9,447.860336,4836.445070,469.509808,10804.821526,306.466520,328.821334,2023


## Change in earnings

In [34]:
def earningsResponse(response_data, corp_noncorp=True):
    ## Create new investment matrix
    inv_base = build_inv_matrix(corp_noncorp)
    inv_ref = build_inv_matrix(corp_noncorp)
    for i in range(96):
        for j in range(57,68):
            if corp_noncorp:
                inv_ref[i,j] = inv_ref[i,j] * (1 + response_results['deltaIc' + str(j + 1960)].tolist()[i])
            else:
                inv_ref[i,j] = inv_ref[i,j] * (1 + response_results['deltaInc' + str(j + 1960)].tolist()[i])
    
    Kstock_base = np.zeros((96,15))
    Kstock_ref = np.zeros((96,15))
    for i in range(96):
        if corp_noncorp:
            Kstock_base[i,3] = np.asarray(base_data['assets_c'])[i]
            Kstock_ref[i,3] = np.asarray(base_data['assets_c'])[i]
        else:
            Kstock_base[i,3] = np.asarray(base_data['assets_nc'])[i]
            Kstock_ref[i,3] = np.asarray(base_data['assets_nc'])[i]
        for j in [56,55,54]:
            Kstock_base[i,j-54] = (Kstock_base[i,j-53] * investmentGfactors_data['pce'][j] / 
                                   investmentGfactors_data['pce'][j+1] - inv_base[i,j]) / (1 - np.asarray(base_data['delta'])[i])
            Kstock_ref[i,j-54] = (Kstock_ref[i,j-53] * investmentGfactors_data['pce'][j] / 
                                  investmentGfactors_data['pce'][j+1] - inv_ref[i,j]) / (1 - np.asarray(base_data['delta'])[i])
        for j in range(57,68):
            Kstock_base[i,j-53] = ((Kstock_base[i,j-54] * (1 - np.asarray(base_data['delta'])[i]) + inv_base[i,j]) * 
                                   investmentGfactors_data['pce'][j+1] / investmentGfactors_data['pce'][j])
            Kstock_ref[i,j-53] = ((Kstock_ref[i,j-54] * (1 - np.asarray(base_data['delta'])[i]) + inv_ref[i,j]) * 
                                  investmentGfactors_data['pce'][j+1] / investmentGfactors_data['pce'][j])
    changeEarnings = np.zeros((96,14))
    for i in range(96):
        for j in range(14):
            if corp_noncorp:
                changeEarnings[i,j] = (Kstock_ref[i,j] - Kstock_base[i,j]) * adjfactor_dep_corp * response_data['deltaEc' + str(j + 2014)][i]
            else:
                changeEarnings[i,j] = (Kstock_ref[i,j] - Kstock_base[i,j]) * adjfactor_dep_noncorp * response_data['deltaEnc' + str(j + 2014)][i]
    newEarnings_total = np.zeros(14)
    for j in range(14):
        newEarnings_total[j] = changeEarnings[:,j].sum()
    earnings_results = pd.DataFrame({'year': range(2014, 2028), 'deltaE': newEarnings_total})
    return earnings_results

In [35]:
earnings_ref_data = earningsResponse(response_results)
earnings_ref_data['earnings_base'] = combined_base['ebitda'].tolist()
earnings_ref_data['ebitda'] = earnings_ref_data['earnings_base'] + earnings_ref_data['deltaE']
#earnings_ref_data.drop(['deltaE', 'earnings_base'], axis=1, inplace=True)
combined_ref = earnings_ref_data.merge(right=taxDep_ref_corp, how='outer', on='year')

In [36]:
earnings_ref_data

,deltaE,year,earnings_base,ebitda
0,0.0,2014,2480.158247,2480.158247
1,0.0,2015,2522.655310,2522.655310
2,0.0,2016,2340.887900,2340.887900
3,0.0,2017,2430.253269,2430.253269
4,0.0,2018,2446.657090,2446.657090
5,0.0,2019,2525.308762,2525.308762
6,0.0,2020,2536.628601,2536.628601
7,0.0,2021,2673.198699,2673.198699
8,0.0,2022,2813.454850,2813.454850
9,0.0,2023,2939.832404,2939.832404


# Recalculate corporate tax using reform parameters

##  NID

In [37]:
def NID_response(capital_path, eta=0.4, id_hc_year=9e99, nid_hc_year=9e99, id_hc_old=0, id_hc_new=0, nid_hc_old=0, nid_hc=0, elast_debt=0):
    # capital_path: growth path of the capital stock
    # eta: retirement rate of existing debt
    # nid_hc: haircut on the net interest deduction, beginning in nid_hc_year
    # id_hc_old, id_hc_new: haircuts on the deduction of interest paid on debt originated before id_hc_year and on debt originated beginning in id_hc_year
    # elast_debt: elastiticity of the debt-to-asset ratio with respect to the present value of the tax shield
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_corp['Kfa'][:57].tolist()
    A = debt_data_corp['A'][:57].tolist()
    L = debt_data_corp['L'][:57].tolist()
    D = [L[i] - A[i] for i in range(len(L))]
    i_t = debt_data_corp['i_t'].tolist()
    i_pr = debt_data_corp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        A.append(A[56] * K_fa[i] / K_fa[56])
        D.append(D[56] * K_fa[i] / K_fa[56])
        L.append(D[i] + A[i])
    ## Apply debt response
    taxshield_base = tau_base
    taxshield_ref = tau_ref * (1 - max(id_hc_new, nid_hc))
    pctchg_delta = elast_debt * (taxshield_ref / taxshield_base - 1)
    D_opt = copy.deepcopy(D)
    L_opt = copy.deepcopy(L)
    for i in range(len(D)):
        if i + 1960 >= nid_hc_year or i + 1960 >= id_hc_year:
            D_opt[i] = D[i] * (1 + pctchg_delta)
            L_opt[i] = D_opt[i] + A[i]
    R = np.zeros(68)
    O = np.zeros(68)
    L2 = copy.deepcopy(L)
    for i in range(1,68):
        R[i] = L2[i-1] * eta
        O[i] = max(L_opt[i] - L2[i-1] * (1 - eta), 0)
        L2[i] = L2[i-1] - R[i] + O[i]
        
    i_a = [x / 100. for x in i_t]
    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_income = [A[i] * i_a[i] for i in range(len(A))]
    int_expense = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            if j + 1960 < id_hc_year and i + 1960 >= id_hc_year:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_old)
            elif j + 1960 >= id_hc_year:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_new)
            else:
                int_expense[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j]
    NID_gross = int_expense - int_income
    NID = np.zeros(len(NID_gross))
    NIP = NID_gross * adjfactor_nid_corp
    for i in range(len(NID)):
        if i + 1960 < nid_hc_year:
            NID[i] = NID_gross[i] * adjfactor_nid_corp
        else:
            NID[i] = NID_gross[i] * adjfactor_nid_corp * (1 - nid_hc)
    NID_results = pd.DataFrame({'year': range(2014,2028), 'nid': NID[54:68]})
    NIP_results = pd.DataFrame({'year': range(2014,2028), 'nip': NIP[54:68]})
    return (NID_results, NIP_results)

tau_nc_base = 0.31782130579911383
tau_nc_ref = 0.31782130579911383
def noncorpIntDeduction_response(capital_path, eta=0.4, id_hc_year=9e99, id_hc_old=0, id_hc_new=0, elast_debt=0):
    Kstock2016 = capital_path['Kstock'][2]
    K_fa = debt_data_noncorp['Kfa'][:57].tolist()
    L = debt_data_noncorp['L'][:57].tolist()
    i_t = debt_data_noncorp['i_t'].tolist()
    i_pr = debt_data_noncorp['i_pr'].tolist()
    for i in range(57,68):
        K_fa.append(K_fa[56] * capital_path['Kstock'][i-54] / Kstock2016)
        L.append(L[56] * K_fa[i] / K_fa[56])
    ##Apply debt response
    taxshield_base = tau_nc_base
    taxshield_ref = tau_nc_ref * (1 - id_hc_new)
    pctchg_delta = (taxshield_ref / taxshield_base - 1) * elast_debt
    L_opt = copy.deepcopy(L)
    for i in range(len(L)):
        if i + 1960 >= id_hc_year:
            L_opt[i] = L[i] * (1 + pctchg_delta)
    R = np.zeros(68)
    O = np.zeros(68)
    L2 = copy.deepcopy(L)
    for i in range(1,68):
        R[i] = L2[i-1] * eta
        O[i] = max(L_opt[i] - L2[i-1] * (1 - eta), 0)
        L2[i] = L2[i-1] - R[i] + O[i]

    i_l = [(i_t[i] + i_pr[i]) / 100. for i in range(len(i_t))]
    int_paid = np.zeros(68)
    int_deducted = np.zeros(68)
    for i in range(1,68):
        for j in range(i+1):
            int_paid[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * adjfactor_id_noncorp
            if j + 1960 < id_hc_year and i + 1960 >= id_hc_year:
                int_deducted[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_old) * adjfactor_id_noncorp
            elif j + 1960 >= id_hc_year:
                int_deducted[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * (1 - id_hc_new) * adjfactor_id_noncorp
            else:
                int_deducted[i] += O[j] * (1 - eta)**(i - j - 1) * i_l[j] * adjfactor_id_noncorp
    ID_results = pd.DataFrame({'year': range(2014,2028), 'intDed': int_deducted[54:68]})
    IP_results = pd.DataFrame({'year': range(2014,2028), 'intpaid': int_paid[54:68]})
    return (ID_results, IP_results)

In [38]:
(NID_ref, NIP_ref) = NID_response(capPath_ref_corp, nid_hc_year=reform_start_year, nid_hc=int_hc, elast_debt=0)
(IntDed_ref_noncorp, IntPaid_ref_noncorp) = noncorpIntDeduction_response(capPath_base_noncorp, id_hc_year=reform_start_year, id_hc_old=int_hc, id_hc_new=int_hc, elast_debt=0)
combined_ref = combined_ref.merge(right=NID_ref, how='outer', on='year')

## Section 199

In [39]:
combined_ref = combined_ref.merge(right=sec199_results, how='outer', on='year')

## Taxable income and tax before credits

In [40]:
combined_ref['taxinc'] = combined_ref['ebitda'] - combined_ref['taxDep'] - combined_ref['nid'] - combined_ref['sec199']
combined_ref['gbc_adj'] = 0.025739617
combined_ref['tau'] = 0.347
combined_ref['tau'][reform_start_year-2014:] = tau_ref - progressive_rate * 0.003
combined_ref['taxbc'] = combined_ref['taxinc'] * (combined_ref['tau'] - combined_ref['gbc_adj'])

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
combined_ref = combined_ref.merge(right=ftc_base, how='outer', on='year')
amt_ref = AMTmodel(ctax_rateChange_year=reform_start_year, ctax_newRate=(tau_ref - progressive_rate * 0.003))
combined_ref = combined_ref.merge(right=amt_ref, how='outer', on='year')
combined_ref['taxrev'] = combined_ref['taxbc'] + combined_ref['amt'] - combined_ref['ftc'] - combined_ref['pymtc']
combined_ref

,deltaE,year,earnings_base,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,0.0,2014,2480.158247,2480.158247,902.336891,231.441998,34.306208,1312.073149,0.02574,0.347,421.517122,101.582376,4.496483,3.700228,320.731000
1,0.0,2015,2522.655310,2522.655310,890.312305,243.981131,34.855388,1353.506487,0.02574,0.347,434.828012,91.794692,4.568463,3.804783,343.797000
2,0.0,2016,2340.887900,2340.887900,827.819233,248.001094,34.816996,1230.250578,0.02574,0.347,395.230772,96.318143,4.563431,3.905060,299.571000
3,0.0,2017,2430.253269,2430.253269,852.830903,268.262930,34.952204,1274.207231,0.02574,0.357,422.094375,99.935946,4.452829,3.991510,322.619749
4,0.0,2018,2446.657090,2446.657090,807.392625,295.580925,35.018974,1308.664567,0.02574,0.357,433.508726,96.600049,4.461335,4.052085,337.317927
5,0.0,2019,2525.308762,2525.308762,787.094657,328.934835,35.349483,1373.929786,0.02574,0.357,455.128507,97.986091,4.503441,4.105823,357.540035
6,0.0,2020,2536.628601,2536.628601,633.603895,362.449185,35.856932,1504.718589,0.02574,0.357,498.453656,103.553749,4.568089,4.158033,395.309963
7,0.0,2021,2673.198699,2673.198699,714.119119,393.699349,36.599743,1528.780487,0.02574,0.357,506.424410,109.003947,4.662722,4.211877,397.871308
8,0.0,2022,2813.454850,2813.454850,785.425745,422.521900,37.604625,1567.902580,0.02574,0.357,519.384009,114.806528,4.790741,4.271076,405.097147
9,0.0,2023,2939.832404,2939.832404,846.027716,448.489428,38.973402,1606.341858,0.02574,0.357,532.117419,121.619274,4.965120,4.339313,411.123952


In [42]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.496483,3.700228,101.582376,421.517122,0.02574,0.347,1312.073149,34.306208,902.336891,231.441998,2480.158247
1,2015,343.797,4.568463,3.804783,91.794692,434.828012,0.02574,0.347,1353.506487,34.855388,890.312305,243.981131,2522.655310
2,2016,299.571,4.563431,3.905060,96.318143,395.230772,0.02574,0.347,1230.250578,34.816996,827.819233,248.001094,2340.887900
3,2017,310.006,4.581153,3.991510,99.935946,409.352303,0.02574,0.347,1274.207231,34.952204,852.830903,268.262930,2430.253269
4,2018,324.343,4.589904,4.068935,96.600049,420.422080,0.02574,0.347,1308.664567,35.018974,807.392625,295.580925,2446.657090
5,2019,343.899,4.633224,4.137342,97.986091,441.389209,0.02574,0.347,1373.929786,35.349483,787.094657,328.934835,2525.308762
6,2020,380.350,4.699734,4.202455,103.553749,483.406470,0.02574,0.347,1504.718589,35.856932,633.603895,362.449185,2536.628601
7,2021,382.662,4.797094,4.267752,109.003947,491.136605,0.02574,0.347,1528.780487,36.599743,714.119119,393.699349,2673.198699
8,2022,389.490,4.928803,4.337259,114.806528,503.704983,0.02574,0.347,1567.902580,37.604625,785.425745,422.521900,2813.454850
9,2023,395.128,5.108207,4.414933,121.619274,516.054000,0.02574,0.347,1606.341858,38.973402,846.027716,448.489428,2939.832404


In [43]:
compare = combined_ref['taxrev'] - combined_base['taxrev']
print ("Change in corporate tax revenue, " + str(reform_start_year) + "-" + str(reform_start_year + 9) + ": " +
       str(int((sum(combined_ref['taxrev'][reform_start_year-2014:reform_start_year-2004]) - 
           sum(combined_base['taxrev'][reform_start_year-2014:reform_start_year-2004])) * 100.) / 100.) + 
       " billion dollars")

Change in corporate tax revenue, 2017-2026: 152.1 billion dollars


In [44]:
for i in compare:
    print i

-5.68434188608e-14
0.0
0.0
12.6137487082
12.974926846
13.6410351901
14.9599628491
15.2093077533
15.6071466628
15.9959524163
16.4880116374
17.0221362662
17.5957801249
18.2315650178


## Calculate percent change in capital income

In [65]:
corpInc_base = capPath_base_corp.drop(['FixedInv', 'FixedK', 'Kstock', 'taxDep'], axis=1, inplace=False)
corpInc_base = corpInc_base.merge(right=NIP_base, how='outer', on='year')
corpInc_base['earnings'] = combined_base['ebitda']
corpInc_base['tax'] = combined_base['taxrev']
corpInc_base['inc_aftertax'] = corpInc_base['earnings'] - corpInc_base['TrueDep'] - corpInc_base['nip'] - corpInc_base['tax']
corpInc_base['cashflow'] = corpInc_base['earnings'] - corpInc_base['Investment'] - corpInc_base['nip'] - corpInc_base['tax']

corpInc_ref = capPath_ref_corp.drop(['FixedInv', 'FixedK', 'Kstock', 'taxDep'], axis=1, inplace=False)
corpInc_ref = corpInc_ref.merge(right=NIP_ref, how='outer', on='year')
corpInc_ref['earnings'] = combined_ref['ebitda']
corpInc_ref['tax'] = combined_ref['taxrev']
corpInc_ref['inc_aftertax'] = corpInc_ref['earnings'] - corpInc_ref['TrueDep'] - corpInc_ref['nip'] - corpInc_ref['tax']
corpInc_ref['cashflow'] = corpInc_ref['earnings'] - corpInc_ref['Investment'] - corpInc_ref['nip'] - corpInc_ref['tax']

In [66]:
indiv_gfactors = corpInc_base.drop(['earnings', 'tax', 'TrueDep', 'cashflow', 'nip'], axis=1, inplace=False)
indiv_gfactors.rename(columns={'inc_aftertax': 'inc_aftertax_base'}, inplace=True)
indiv_gfactors['inc_aftertax_ref'] = corpInc_ref['inc_aftertax']
indiv_gfactors['equity'] = indiv_gfactors['inc_aftertax_ref'] / indiv_gfactors['inc_aftertax_base']
corpshare_totalint = 1.0  ##Corporate interest share of all interest
indiv_gfactors['debt'] = 1 + (corpInc_ref['nip'] / corpInc_base['nip'] - 1) * corpshare_totalint

## Calculate percent change in pass-through income

In [67]:
partner_data = pd.read_csv(ctax_data_path + 'partnership data.csv')
Scorp_data = pd.read_csv(ctax_data_path + 'Scorp data.csv')
sp_data = pd.read_csv(ctax_data_path + 'sp_nonfarm data.csv')

In [68]:
## Calculate shares for 2013 to distribute changes in depreciation and interest paid
totaldep = partner_data['dep_total'][19] + Scorp_data['dep_total'][18] + sp_data['dep_total'][16]
depshare_scorp_posinc = Scorp_data['dep_posinc'][18] / totaldep
depshare_scorp_neginc = Scorp_data['dep_total'][18] / totaldep - depshare_scorp_posinc
depshare_sp_posinc = sp_data['dep_posinc'][16] / totaldep
depshare_sp_neginc = sp_data['dep_total'][16] / totaldep - depshare_sp_posinc
depshare_partner_posinc = partner_data['dep_posinc'][19] / totaldep
depshare_partner_neginc = partner_data['dep_total'][19] / totaldep - depshare_partner_posinc

totalint_exfin = (partner_data['intpaid_total'][19] + Scorp_data['intpaid_total'][18] + sp_data['mortintpaid'][16] + 
                  sp_data['otherintpaid'][16] - partner_data['intpaid_fin_total'][16] - Scorp_data['intpaid_fin'][16] - 
                  sp_data['mortintpaid_fin'][16] - sp_data['otherintpaid_fin'][16])
intshare_scorp_posinc = (Scorp_data['intpaid_posinc'][18] - Scorp_data['intpaid_fin_posinc'][18]) / totalint_exfin
intshare_scorp_neginc = (Scorp_data['intpaid_total'][18] - Scorp_data['intpaid_total'][18]) / totalint_exfin - intshare_scorp_posinc
intshare_sp_posinc = (sp_data['mortintpaid_posinc'][16] + sp_data['otherintpaid_posinc'][16] - 
                      sp_data['mortintpaid_fin_posinc'][16] - sp_data['otherintpaid_fin_posinc'][16]) / totalint_exfin
intshare_sp_neginc = (sp_data['mortintpaid'][16] + sp_data['otherintpaid'][16] - sp_data['mortintpaid_fin'][16] - 
                      sp_data['otherintpaid_fin'][16]) / totalint_exfin - intshare_sp_posinc
intshare_partner_posinc = (partner_data['intpaid_posinc'][19] - partner_data['intpaid_fin_posinc'][19]) / totalint_exfin
intshare_partner_neginc = (partner_data['intpaid_total'][19] - partner_data['intpaid_fin_total'][19]) / totalint_exfin - intshare_partner_posinc

In [69]:
## Construct Sch C income
sp_posinc = [sp_data['netinc'][17]]
sp_neginc = [sp_data['netloss'][17]]
for i in range(55,68):
    sp_posinc.append(sp_posinc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
    sp_neginc.append(sp_neginc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
SchC_results = pd.DataFrame({'year': range(2014,2028), 'netinc_pos_base': sp_posinc, 'netinc_neg_base': sp_neginc})
SchC_results['intpaid_pos_base'] = IntPaid_base_noncorp['intpaid'] * intshare_sp_posinc
SchC_results['intpaid_neg_base'] = IntPaid_base_noncorp['intpaid'] * intshare_sp_neginc
SchC_results['dep_pos_base'] = taxDep_base_noncorp['taxDep'] * depshare_sp_posinc
SchC_results['dep_neg_base'] = taxDep_base_noncorp['taxDep'] * depshare_sp_neginc
SchC_results['ebitda_pos_base'] = SchC_results['netinc_pos_base'] + SchC_results['intpaid_pos_base'] + SchC_results['dep_pos_base']
SchC_results['ebitda_neg_base'] = -SchC_results['netinc_neg_base'] + SchC_results['intpaid_neg_base'] + SchC_results['dep_neg_base']

## Construct partnership income
partner_posinc = [partner_data['netinc_total'][20]]
partner_neginc = [partner_data['netloss_total'][20]]
for i in range(55,68):
    partner_posinc.append(partner_posinc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
    partner_neginc.append(partner_neginc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
partner_results = pd.DataFrame({'year': range(2014,2028), 'netinc_pos_base': partner_posinc, 'netinc_neg_base': partner_neginc})
partner_results['intpaid_pos_base'] = IntPaid_base_noncorp['intpaid'] * intshare_partner_posinc
partner_results['intpaid_neg_base'] = IntPaid_base_noncorp['intpaid'] * intshare_partner_neginc
partner_results['dep_pos_base'] = taxDep_base_noncorp['taxDep'] * depshare_partner_posinc
partner_results['dep_neg_base'] = taxDep_base_noncorp['taxDep'] * depshare_partner_neginc
partner_results['ebitda_pos_base'] = partner_results['netinc_pos_base'] + partner_results['intpaid_pos_base'] + partner_results['dep_pos_base']
partner_results['ebitda_neg_base'] = -partner_results['netinc_neg_base'] + partner_results['intpaid_neg_base'] + partner_results['dep_neg_base']

## Construct S corporation income
scorp_posinc = [Scorp_data['netinc_total'][18] * investmentGfactors_data['prop_inc'][54] / investmentGfactors_data['prop_inc'][53]]
scorp_neginc = [Scorp_data['netloss_total'][18] * investmentGfactors_data['prop_inc'][54] / investmentGfactors_data['prop_inc'][53]]
for i in range(55,68):
    scorp_posinc.append(scorp_posinc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
    scorp_neginc.append(scorp_neginc[0] * investmentGfactors_data['prop_inc'][i] / investmentGfactors_data['prop_inc'][54])
Scorp_results = pd.DataFrame({'year': range(2014,2028), 'netinc_pos_base': scorp_posinc, 'netinc_neg_base': scorp_neginc})
Scorp_results['intpaid_pos_base'] = IntPaid_base_noncorp['intpaid'] * intshare_scorp_posinc
Scorp_results['intpaid_neg_base'] = IntPaid_base_noncorp['intpaid'] * intshare_scorp_neginc
Scorp_results['dep_pos_base'] = taxDep_base_noncorp['taxDep'] * depshare_scorp_posinc
Scorp_results['dep_neg_base'] = taxDep_base_noncorp['taxDep'] * depshare_scorp_neginc
Scorp_results['ebitda_pos_base'] = Scorp_results['netinc_pos_base'] + Scorp_results['intpaid_pos_base'] + Scorp_results['dep_pos_base']
Scorp_results['ebitda_neg_base'] = -Scorp_results['netinc_neg_base'] + Scorp_results['intpaid_neg_base'] + Scorp_results['dep_neg_base']

In [70]:
## Calculate new EBITDA
earnings_results_noncorp = earningsResponse(response_results, False)
earnings_results_noncorp['ebitda_base'] = (SchC_results['ebitda_pos_base'] + SchC_results['ebitda_neg_base'] +
                                           partner_results['ebitda_pos_base'] + partner_results['ebitda_neg_base'] +
                                           Scorp_results['ebitda_pos_base'] + Scorp_results['ebitda_neg_base'])
earnings_results_noncorp['ebitda_ref'] = earnings_results_noncorp['ebitda_base'] + earnings_results_noncorp['deltaE']
earnings_results_noncorp['ebitda_chgfactor'] = earnings_results_noncorp['ebitda_ref'] / earnings_results_noncorp['ebitda_base']

SchC_results['ebitda_pos_ref'] = SchC_results['ebitda_pos_base'] * earnings_results_noncorp['ebitda_chgfactor']
SchC_results['ebitda_neg_ref'] = SchC_results['ebitda_neg_base'] * earnings_results_noncorp['ebitda_chgfactor']
partner_results['ebitda_pos_ref'] = partner_results['ebitda_pos_base'] * earnings_results_noncorp['ebitda_chgfactor']
partner_results['ebitda_neg_ref'] = partner_results['ebitda_neg_base'] * earnings_results_noncorp['ebitda_chgfactor']
Scorp_results['ebitda_pos_ref'] = Scorp_results['ebitda_pos_base'] * earnings_results_noncorp['ebitda_chgfactor']
Scorp_results['ebitda_neg_ref'] = Scorp_results['ebitda_neg_base'] * earnings_results_noncorp['ebitda_chgfactor']

In [71]:
## Recalculate sole proprietorship net income or loss
SchC_results['dep_pos_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_sp_posinc
SchC_results['dep_neg_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_sp_neginc
SchC_results['intpaid_pos_ref'] = IntDed_ref_noncorp['intDed'] * intshare_sp_posinc
SchC_results['intpaid_neg_ref'] = IntDed_ref_noncorp['intDed'] * intshare_sp_neginc
SchC_results['netinc_pos_ref'] = SchC_results['ebitda_pos_ref'] - SchC_results['dep_pos_ref'] - SchC_results['intpaid_pos_ref']
SchC_results['netinc_neg_ref'] = SchC_results['ebitda_neg_ref'] - SchC_results['dep_neg_ref'] - SchC_results['intpaid_neg_ref']

## Recalculate partnership net income or loss
partner_results['dep_pos_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_partner_posinc
partner_results['dep_neg_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_partner_neginc
partner_results['intpaid_pos_ref'] = IntDed_ref_noncorp['intDed'] * intshare_partner_posinc
partner_results['intpaid_neg_ref'] = IntDed_ref_noncorp['intDed'] * intshare_partner_neginc
partner_results['netinc_pos_ref'] = partner_results['ebitda_pos_ref'] - partner_results['dep_pos_ref'] - partner_results['intpaid_pos_ref']
partner_results['netinc_neg_ref'] = partner_results['ebitda_neg_ref'] - partner_results['dep_neg_ref'] - partner_results['intpaid_neg_ref']

## Recalculate S corporation net income or loss
Scorp_results['dep_pos_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_scorp_posinc
Scorp_results['dep_neg_ref'] = taxDep_ref_noncorp['taxDep'] * depshare_scorp_neginc
Scorp_results['intpaid_pos_ref'] = IntDed_ref_noncorp['intDed'] * intshare_scorp_posinc
Scorp_results['intpaid_neg_ref'] = IntDed_ref_noncorp['intDed'] * intshare_scorp_neginc
Scorp_results['netinc_pos_ref'] = Scorp_results['ebitda_pos_ref'] - Scorp_results['dep_pos_ref'] - Scorp_results['intpaid_pos_ref']
Scorp_results['netinc_neg_ref'] = Scorp_results['ebitda_neg_ref'] - Scorp_results['dep_neg_ref'] - Scorp_results['intpaid_neg_ref']

In [72]:
indiv_gfactors['SchC_pos'] = SchC_results['netinc_pos_ref'] / SchC_results['netinc_pos_base']
indiv_gfactors['SchC_neg'] = SchC_results['netinc_neg_ref'] / SchC_results['netinc_neg_base'] * -1
indiv_gfactors['e26270_pos'] = ((partner_results['netinc_pos_ref'] + Scorp_results['netinc_pos_ref']) / 
                                (partner_results['netinc_pos_base'] + Scorp_results['netinc_pos_base']))
indiv_gfactors['e26270_neg'] = ((partner_results['netinc_neg_ref'] + Scorp_results['netinc_neg_ref']) / 
                                (partner_results['netinc_neg_base'] + Scorp_results['netinc_neg_base']) * -1)

In [74]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1177.556186,2014,1419.754012,1419.754012,1.000000,1.0,1.0,1.0,1.0,1.0
1,1163.657496,2015,1353.252325,1353.252325,1.000000,1.0,1.0,1.0,1.0,1.0
2,974.598908,2016,1154.118239,1154.118239,1.000000,1.0,1.0,1.0,1.0,1.0
3,1010.486042,2017,1174.425426,1161.811678,0.989260,1.0,1.0,1.0,1.0,1.0
4,1052.746832,2018,1110.664853,1097.689926,0.988318,1.0,1.0,1.0,1.0,1.0
5,1090.878874,2019,1097.439760,1083.798724,0.987570,1.0,1.0,1.0,1.0,1.0
6,1128.498094,2020,1000.038396,985.078433,0.985041,1.0,1.0,1.0,1.0,1.0
7,1171.004829,2021,1064.506120,1049.296812,0.985712,1.0,1.0,1.0,1.0,1.0
8,1217.284475,2022,1130.197943,1114.590797,0.986191,1.0,1.0,1.0,1.0,1.0
9,1265.620646,2023,1185.032349,1169.036396,0.986502,1.0,1.0,1.0,1.0,1.0


## Pass changes to Tax-Calculator

In [75]:
records_url = 'C:/Users/cody.kallen/My Documents/GitHub/tax-calculator/puf.csv'
def make_calculator(reform_dict, start_year):
    policy1 = Policy()
    behavior1 = Behavior()
    records1 = Records(records_url)
    if reform_dict != {}:
        policy1.implement_reform(reform_dict)
    calc1 = Calculator(records = records1, policy = policy1, behavior = behavior1)
    for i in range(start_year - 2013):
        calc1.increment_year()
    assert calc1.current_year == start_year
    calc1.calc_all()
    return(calc1)

calc_base = make_calculator({}, 2014)
calc_ref = make_calculator({}, 2014)

indiv_rev_impact = np.zeros(14)
for i in range(2014,2027):
    calc_ref2 = copy.deepcopy(calc_ref)
    ## Change business income
    calc_ref2.records.e00900p = np.where(calc_ref2.records.e00900p >= 0, 
                                         calc_ref2.records.e00900p * indiv_gfactors['SchC_pos'][i-2014], 
                                         calc_ref2.records.e00900p * indiv_gfactors['SchC_neg'][i-2014])
    calc_ref2.records.e00900s = np.where(calc_ref2.records.e00900s >= 0, 
                                         calc_ref2.records.e00900s * indiv_gfactors['SchC_pos'][i-2014], 
                                         calc_ref2.records.e00900s * indiv_gfactors['SchC_neg'][i-2014])
    calc_ref2.records.e00900 = np.where(calc_ref2.records.e00900 >= 0, 
                                        calc_ref2.records.e00900 * indiv_gfactors['SchC_pos'][i-2014], 
                                        calc_ref2.records.e00900 * indiv_gfactors['SchC_neg'][i-2014])
    change_e26270 = np.where(calc_ref2.records.e26270 >= 0, 
                             calc_ref2.records.e26270 * (indiv_gfactors['e26270_pos'][i-2014] - 1), 
                             calc_ref2.records.e26270 * (indiv_gfactors['e26270_neg'][i-2014] - 1))
    calc_ref2.records.e26270 = calc_ref2.records.e26270 + change_e26270
    calc_ref2.records.e02000 = calc_ref2.records.e02000 + change_e26270
    ## Change investment income
    calc_ref2.records.e00600 = calc_ref2.records.e00600 * indiv_gfactors['equity'][i-2014]
    calc_ref2.records.e00650 = calc_ref2.records.e00650 * indiv_gfactors['equity'][i-2014]
    calc_ref2.records.p22250 = calc_ref2.records.p22250 * indiv_gfactors['equity'][i-2014]
    calc_ref2.records.p23250 = calc_ref2.records.p23250 * indiv_gfactors['equity'][i-2014]
    
    calc_base.calc_all()
    calc_ref2.calc_all()
    indiv_rev_impact[i-2014] = sum((calc_ref2.records.combined - calc_base.records.combined) * calc_base.records.s006) / 10**9
    if i < 2026:
        calc_base.increment_year()
        calc_ref.increment_year()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [76]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

0.0
6.97103973625e-14
0.0
-3.56217590717
-3.9195359415
-4.18888987803
-5.14175029962
-5.04644988941
-5.02935627404
-5.097470419
-5.22358180026
-5.36964499614
-5.56539538412
0.0
-48.1442507893


In [77]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,0.000000e+00,2014,-5.684342e-14,-5.684342e-14
1,6.971040e-14,2015,0.000000e+00,6.971040e-14
2,0.000000e+00,2016,0.000000e+00,0.000000e+00
3,-3.562176e+00,2017,1.261375e+01,9.051573e+00
4,-3.919536e+00,2018,1.297493e+01,9.055391e+00
5,-4.188890e+00,2019,1.364104e+01,9.452145e+00
6,-5.141750e+00,2020,1.495996e+01,9.818213e+00
7,-5.046450e+00,2021,1.520931e+01,1.016286e+01
8,-5.029356e+00,2022,1.560715e+01,1.057779e+01
9,-5.097470e+00,2023,1.599595e+01,1.089848e+01


In [78]:
for i in range(3,13):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

9.05157280107
9.05539090454
9.45214531204
9.81821254943
10.1628578639
10.5777903888
10.8984819973
11.2644298371
11.6524912701
12.0303847408


103.963757665
